# Setting up scheduled queries for Big Query on GCP

# Introduction

Why would we want to schedule a query, etc.

While creating a scheduled query is quite easy, there are a few things you need to consider when making and deploying a scheduled query for production. This article will go over different ways of setting up scheduled queries and how well each is suited for production ready systems.

## What is needed of a production ready scheduled query deployment

1. **The schedule of the query.**
This one might seem obvious, but if we can't define at what time the query is to run, we can't schedule a query

2. **First run of the query.**
If you think this is taken care of in the schedule, you are might risk getting some odd reports. If you want weekly averages calculated every Friday, keyed to the weeknumber. But your first query run happens on a Wednesday, not only will you have an average that spans a different set of days, come Friday, you will have 2 aggregates for the same week, but with different numbers.

3. **Query and deployment as code, in seperate files.**
Your devOps script that can deploy your query, so that the same thing can be deployed across testing, development and production environments and your developer wants to be able to edit the query in its own `.sql` file, so that their development tools can assist them, and so they don't have to touch deployment code when they are developing the query.

4. **Backfill, historical runs.**
You need to be able to re-run queries for past dates. If something went wrong in production and the scheduled query didn't have access to all numbers on Friday, you want to be able to re-run it on Monday *as if it was Friday*. You might also want to be able to create a report for Friday last year - long before the query was scheduled.

5. **Control the schema of the destination table.**
If the results produced by the scheduled query are large, and expected to run for a long time, the destination table should be 

## The data

For this tutorial, we will imagine we are in the business of lightning strikes and management has decided that weekly aggregates of lightning strikes are critical to growth going forward.

We are also going to imagine that aggregating this data is an operation that is so heavy, that it doesn't make sense to do it on the fly.

The [National Oceanic and Atmospheric Administration (NOAA)](https://www.noaa.gov) publishes times and locations of all observed lightning strikes, and this is available as a public dataset in BigQuery. The dataset is licensed through [Vaisala](https://www.vaisala.com/en/products/data-subscriptions-and-reports/data-sets/nldn) and according to Google is [freely available to the public, under data.gov license](http://www.data.gov/privacy-policy#data_policy). The data can be accessed in BigQuery as `bigquery-public-data:noaa_lightning`-dataset.

To play with it yourself, or any of the other many public datasets available in BigQuery, open the console, click `+ ADD DATA`>`Pin Project` and pin "*bigquery-public-data*".

## The Query of interest

As a start, we use the following query, but we will expand it, as it becomes clear that this is too simple for a production environment

```SQL
SELECT
  SUM(number_of_strikes) as n_strikes
FROM
  `bigquery-public-data.noaa_lightning.lightning_*`
WHERE
  day BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 week)
  AND CURRENT_DATE()
```

This query will return the sum of lightning strikes that happened during each lightningstrike observation between the date that the query was run, and 1 week before the query was run. If we run this weekly, we will build up a table of weekly averages. If we run it daily, we will build up a table of rolling, weekly averages.

*A note about the asterisk* The lightning table is partitioned into several tables. By querying from `lightning_*` we tell Big Query to look at all the lightning tables, instead of just the table with 2019 data (if we wrote `lightning_2019`). You can uptimize the query here, since we normally won't need to look at 2017 data to do a job on 2019 data. But just around new year, we might have queries that do span 2 tables, so be careful how you optimize here. In this article, I have gone with the most straightforward solution.

## The Schedule

We want to run this every Sunday morning, 00:00Z and aggregate the past week

## The Raw data
The raw data looks like the following, and can be plotted on a map using BigQuery Geoviz

![](assets/lighning_strike_raw_viz.png)

## The Destination table

The final result will be a table that displays the weekly number of lightning strikes. Each additional row is generated by the end of the week and added to the table. For this data, such a simple query can be made on the fly, but if we imagine that the results are much more complex and may span hundreds of of row for each week, you can hopefully imagine why you'd want to do it on a schedule, rather than on the fly.

![](assets/destination_table_preview.png)

---
# Using the Bigquery Console

This is by far the most straight-forward way of working with, and trying out features and services in GCP, and it is a nice place to start, as you get a great IDE for writing and testing your query.

You simply write your query and and click create scheduled query and choose when and how often you want it to run.

![Screenshot of user about to create Schedule query in GCP Console](assets/create_scheduled_query.png)

You can run the query before you set it up, and you can easily get an overview of most of the settings available to you. You should definitely start every prototype here.

![Screenshot of user filling out the settings of a scheduled query in the GCP Console](assets/scheduled_query_options.png)

Clicking "Schedule" will run the query, create a new table, and append the result to that table. Easy as that.

However, neither the query, nor the setup of the query are defined as code and version controlled. This is not good for reproduceability, further development or any kind if production environment.

There are some other things that aren't quite right with this query yet. When we get to week two, we will have a hard time identifying what time period a given result belongs to. Moreover, we are only able to calculate aggregates into the future. What if we want aggregates from dates *before* we deployed the scheduled query?

## Adding Backfill

Backfill, or historical queries, is to run a scheduled query with a date set in the past. This is usefull if a query fail and you need to re-run it, or if you need the query to be run *last* week, instead of just next week.

If we instead of using `CURRENT_DATE()`-function to select for the last 7 days, we use the `@run_date` parameter, we can ask BigQuery to do a manual run of the scheduled query with `@run_date` substituted for any date we like, so our SQL will now look like this:

```SQL
SELECT
  SUM(number_of_strikes),
  @run_date as last_week_from
FROM
  `bigquery-public-data.noaa_lightning.lightning_*`
WHERE
  day BETWEEN DATE_SUB(@run_date, INTERVAL 1 week)
  AND @run_date
```

I also added the date as a column, so results from different runs can be identified.

Note that once the query is parameterised, you can no longer execute it from the console. Checks to see if the query is still valid will also often fail, making it difficult to update the query.

---
# Using the Cloud SDK

We also want to store our SQL code in VCS and deploy the query as part of our CD/CI-pipeline. For this, it is not feasible to have a developer log on to the webconsole and manually do deployments. So we need to move away from the webconsole, in order to test the parameterised query as well as having deployment script that DevOps can automate.

One way is to use the Cloud SDK from the shell, e.g. the `gcloud` and the `bq` shell commands. Often these are the easiest ways to interact with the GCP.

According to the documentations there are 2 different commands to deploy scheduled queries, and it is not immediately clear which is preferred:

1. `bq query` followed by a bunch of flags and finally the query
2. `bq mk --transfer_config`, hooks more directly into the [BigQuery Transfer service](https://cloud.google.com/bigquery-transfer/docs/), which Scheduled Queries are a part of. Uses a mix of commandline flags and JSON.

## Using `bq query`

![](assets/bq_query_header.png)

Most of the flags are fairly self explanatory, and our deployment command will roughly be the following, but needs a few more refinements.

```Bash
bq query \
    --use_legacy_sql=false \
    --destination_table=lightning.test \
    --display_name='Weekly Lightning' \
    --append_table \
    --schedule='every sun 00:00' \
'SELECT
  SUM(number_of_strikes),
  @run_date as last_week_from
FROM
  `bigquery-public-data.noaa_lightning.lightning_*`
WHERE
  day BETWEEN DATE_SUB(@run_date, INTERVAL 1 week)
  AND @run_date'
```

### Remember the schedule flag

Note that if you forget the `--schedule`-flag (as [Google did in their example code](https://cloud.google.com/bigquery/docs/scheduling-queries#setting_up_a_scheduled_query), at the time of writing) you will be met with a very cryptic error:

```
Error in query string: Error processing job 'scheduled-queries:bqjob_r3159aed97aa830f5_0000016dd9faa75c_1':
Undeclared parameter 'run_date' is used assuming different types (DATE vs INT64) at [3:3]
```

This is because without the `--schedule` flag, the `@run_time` SQL-parameter will not have a value, so BigQuery will fail at type-checking your query. It first assumes the empty parameter must be a INT64 in line 3, and then later sees that it is used as a date.

This is the same error that causes you to be unable to save edits this query in the webconsole.

### The interval format of the schedule flag

Another under-documented feature here is what values are accepted by the `--schedule` flag. It follows the [App-engine flexible environment's cron.yaml-syntax](https://cloud.google.com/appengine/docs/flexible/python/scheduling-jobs-with-cron-yaml#the_schedule_format). The easiest way to create these is to use the console to create the desired  scheduled with point and click, and then  inspect the final result in the completed schedule under the configuration tab.

### Don't use false for either append or replace flags

Both the `--append_table` and `--replace` are shown in the documentation as taking a `true` value. This is strictly not necessary, you don't need to write `--replace=true` if you want to truncate the table everytime the job is run, and if you don't want the table truncated, you actually cannot write `--replace=False`.

- Add the `--replace` flag for replacing tables (What BQ Transfer internally calls WRITE_TRUNCATE as your write_disposition value)
- Add the `--append_table` flag if you want to append to the destination table (What BQ Transfer internally calls WRITE_APPEND as your write_disposition value)
- If you add both flags or none of them, you will append.

## Working with an external .sql file

But of course we want the deployment command seperated from the .sql file. Let us see what we can do in BASH:

```bash
# deploy_query.sh

bq query \               
    --use_legacy_sql=false \ 
    --destination_table=lightning.test \
    --display_name='Weekly Lightning' \            
    --append_table \
    --schedule='every sun 00:00' \                
    `cat weekly.sql`
```

## Backfill

To run the query on historical dates looks fairly simple

```bash
bq mk \
    --transfer_run \
    --start_time='start_time' \
    --end_time='end_time' \
    [resource_name]
```

However, we don't know the name of the newly created query ressource! 

The ressource name is returned by the `bq query` command on success and looks something like

```bash
Transfer configuration 'projects/655462874078/locations/us/transferConfigs/5dafef56-0000-2a29-a894-f4f5e80c659c' successfully created.
```

and we can cut it out with a bit of more bash

```bash
# deploy_query.sh

bq query \               
    --use_legacy_sql=false \ 
    --destination_table=lightning.test \
    --display_name='Weekly Lightning' \            
    --append_table \
    --schedule='every sun 00:00' \                
    `cat weekly.sql`| tee /dev/tty | cut -d "'" -f 2 > resource_name.txt
```

Here we pipe the result of the query deployment to [tee](https://en.wikipedia.org/wiki/Tee_(command)), where we can save it to our logs (or in this example, just print it to the terminal), and then pipe the original result to the [cut](https://en.wikipedia.org/wiki/Cut_(Unix)) utility, which will cut it into fields on the single quotation mark, and we save the second field to the filesystem.

```bash
# backfill.sh

start_time=@1
end_time=@2

bq mk \
    --transfer_run \
    --start_time=$start_time \
    --end_time=$end_time \
    `cat resource_name.txt`
```

## Reviewing `bq query` way of deployment

Using Google's Cloud SDK we can create a deployment with seperate SQL and deployment files that can be stored in version control and deploy on different environments. We can run backfill on historical dates.

We can not control the schema of the destination table. One workaround is to create the table in a seperate command in `deploy_query.sh`, but this requires that the developer to write the schema by hand. Not impossible, but combersome and requires a lot of testing.

We also cannot control when the first query is run. With `bq query ...` command, the query will run immediately and save rows to the destination table. A workaround could be to follow the deployment with a query that deletes all rows after the first run, so that over time, only data from the scheduled events (and backfills) will exist in the destination table. Also not impossible, but not particularly elegant.

## Using `bq mk --transfer_config`
![](assets/bq_mk_transfer_config_header.png)

This command mimics more directly the transfer service API. Most flags corrosponds to a field in the [TransferConfig Resource](https://cloud.google.com/bigquery-transfer/docs/reference/datatransfer/rest/v1/projects.locations.transferConfig), although this isn't completely one-to-one.

Moreover, this command uses a mix of commandline flags and in-line JSON. From the perspective of someone who wants to deploy a scheduled query, it is not obvious what kind of information goes into the JSON and what should be a commandline flag. But roughly speaking the JSON describes the table, and the flags describes the schedule. In this context, the query also describes the layout of the table, as the result of a query is a table. An overview of all available parameters are not documented anywhere.

Let us start with a simplified command, and try and expand it.

```bash
bq mk --transfer_config \
    --target_dataset=lightning \
    --display_name='Weekly Lightning' \
    --params='{"query":"SELECT 1 from mydataset.test","destination_table_name_template":"test","write_disposition":"WRITE_APPEND", "partitioning_field": "last_week_from"}' \
    --schedule=schedule-string \
    --schedule_start_time=datetime \
    --data_source=scheduled_query
```

The documentation for all the flags and parameters used are scattered between the `bq mk --help` command, the [scheduled query documentation page](https://cloud.google.com/bigquery/docs/scheduling-queries) and the [TransferConfig Resource documentaion page](https://cloud.google.com/bigquery-transfer/docs/reference/datatransfer/rest/v1/projects.locations.transferConfig).


*Note* 
- `--data_source` flag *must* be "scheduled_query", or else we are not creating a scheduled query.
- `--schedule_start_time`: "Time to start scheduling transfer runs for the given transfer configuration. If empty, the default value for the start time will be to run immediately. The format for the time stamp is [RFC3339 UTC "Zulu".](https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp)" - manpage for `bq mk`

### Start time

If we don't supply a start time, the query will run as soon as we deploy it. If we deploy on Thursday, we will suddenly have a summary for the wrong timespan, since we expect weekly summaries Sunday-Sunday.

We can change when the first run of the schedule is, by using the `--schedule_start_time`-flag. However, unlike its name or man-page description, this flag doesn't specify *when* the first run is, it specifies a time which the schedule must run no sooner than. Simply supplying the current time will get you what you want. Adding the current time is fine, just remember to convert it to UTC.

If you are on a BSD system (such as MacOS) or a GNU system (such as Linux) the `date` program will behave very differently, but for displaying the current time in UTC "Zulu", it works out just the same.

```bash
date -u +%Y-%m-%dT%TZ
```

### Partitioning the destination table

It is generally a good idea to partition your tables. This makes it possible to drastically reduce the size of your queries later on. This is not possible to do, when using the `bq query`-command. Which column to partition on is defined in the `partitioning_field`-field inside the JSON parameters.

### Splitting out the query and writing the deployment files

In terms of splitting this command into a SQL-file, a deployment script and a backfill script, the major hurdly is to insert the SQL query string into the JSON string and the JSON string into the `bq mk` command.  (we'll tackle `--schedule_start_time` a little later)

If we naively `cat` the sql-query directly into the json string,

```bash
...
    --params='{"query":"'`cat weekly.sql`'","destination_table_name_template":"test2","write_disposition":"WRITE_APPEND"}' \
...
```

We will get a FATAL error

```
FATAL Flags positioning error: Flag '--schedule=every' appears after final command line argument. Please reposition the flag.
Run 'bq help' to get help.
```

This is because of the line breaks that `cat` prints from the file. In bash, if you echo a variable with quotes, it will retain new lines, but if you don't quote the variable, it will have newlines removed. Using this, we can split the loading of the query and the json construction into seperate operation and get:

```bash
# deploy_query-transfer_config.sh

query=`cat weekly.sql`
json='{
"destination_table_name_template" : "test",
"write_disposition": "WRITE_APPEND",
"partitioning_field": "last_week_from",
"query": "'${echo $query}'"
}'

bq mk --transfer_config \
    --target_dataset=lightning \
    --display_name='Weekly Lightning' \
    --schedule='every sunday 00:00' \
    --schedule_start_time="`date -u +%Y-%m-%dT%TZ`" \
    --data_source=scheduled_query \
    --params="$json" | tee /dev/tty | cut -d "'" -f 2 > resource_name.txt
```

This will also work with quotes inside of the SQL query. Here we also store the resource name, just like we did with the `bq query` command, and backfill can be done just like earlier.

### Backfill

This is done just as we did previously.

# Links

- [Scheduled Queries documentation](https://cloud.google.com/bigquery/docs/scheduling-queries)
- [BigQuery Transfer documentation](https://cloud.google.com/bigquery-transfer/docs/working-with-transfers)
- [BigQuery Transfer's client library documentation](https://cloud.google.com/bigquery-transfer/docs/reference/libraries#client-libraries-usage-python)